# Setup

In [ ]:
from os import environ


environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/workspaces/freespeech/id/test-service-credentials.json"
environ["DEEPGRAM_TOKEN"] = ""
environ["NOTION_TOKEN"] = ""

OUTPUT_DIR = "/workspaces/freespeech/output"

# Download Video

In [ ]:
from freespeech.lib import youtube


VIDEO_URL = "https://youtu.be/bhRaND9jiOA"
audio_file, video_file, meta, subtitles = youtube.download(VIDEO_URL, output_dir=OUTPUT_DIR)

# Cut Video

In [ ]:
from freespeech.lib import media


START = '00:00:00'
FINISH = '00:00:15'

cut_audio_file = await media.cut(audio_file, start=START, finish=FINISH, output_dir=OUTPUT_DIR)
cut_video_file = await media.cut(video_file, start=START, finish=FINISH, output_dir=OUTPUT_DIR)

cut_video_with_audio_file = await media.dub(cut_video_file, cut_audio_file, output_dir=OUTPUT_DIR)

# Load Transcript
## From Notion Page

In [ ]:
from freespeech.lib import notion
from freespeech.lib import language


# Get the GUID from the share link in Notion for database and page
database_id = ""
target_page_id = ""

target_transcript = await notion.get_transcript(target_page_id)

# Transcribe

In [ ]:
from freespeech.lib import media, speech
from freespeech.lib.storage import obj

audio_file = cut_audio_file

audio_file_mono = await media.multi_channel_audio_to_mono(audio_file, output_dir=OUTPUT_DIR)
(audio_info, *_), _ = media.probe(audio_file_mono)

audio_uri = f"gs://freespeech-tests/transcribe/{audio_file_mono.name}"
await obj.put(audio_file_mono, audio_uri)

transcript = await speech.transcribe(audio_uri, audio_info, "ru-RU", "default", provider="Deepgram")
normalized_events = speech.normalize_speech(transcript, gap_ms=2000, length=500)


# Translate

In [ ]:
source_transcript = await notion.get_transcript(str(target_transcript.source))
translated_events = language.translate_events(
    source_transcript.events,
    source=source_transcript.lang,
    target=target_transcript.lang,
)

updated_transcript = await notion.put_transcript(
    database_id=database_id, transcript=replace(target_transcript, events=translated_events)
)